In [39]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split

from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams

# Gets the box score of every single game (NBA + WNBA + others: nba.com doesn't seperate the leagues) 
# from 2014-2021 into a dataframe.
all_games_finder = leaguegamefinder.LeagueGameFinder()
all_games = all_games_finder.get_data_frames()[0]

# Creates empty dataframe with the same column names passed in from the dataframe containing all the games.
column_names = all_games.columns.values
games = pd.DataFrame(columns = column_names)

# Appends every game containing an NBA team in the all_games df to the games df. 
# Note: this double loop might legit get us 50% off... need to come up with a better implementation lmao. 
# It takes like 10 minutes... 
nba_teams = teams.get_teams()
for team in nba_teams:
    temp_id = team['id']
    for index, row in all_games.iterrows():
        if row['TEAM_ID'] == temp_id:
            games = games.append(row, ignore_index = True)
    
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612737,ATL,Atlanta Hawks,0022100066,2021-10-28,ATL @ WAS,L,241,111,...,0.643,6,37,43,26,4,3,13,16,-11.0
1,22021,1610612737,ATL,Atlanta Hawks,0022100059,2021-10-27,ATL @ NOP,W,240,102,...,0.824,21,34,55,21,4,4,11,14,3.0
2,22021,1610612737,ATL,Atlanta Hawks,0022100043,2021-10-25,ATL vs. DET,W,238,122,...,0.857,10,39,49,24,11,3,13,19,18.0
3,22021,1610612737,ATL,Atlanta Hawks,0022100027,2021-10-23,ATL @ CLE,L,241,95,...,0.600,17,37,54,20,5,3,9,23,-6.0
4,22021,1610612737,ATL,Atlanta Hawks,0022100014,2021-10-21,ATL vs. DAL,W,242,113,...,0.889,6,49,55,31,8,9,13,16,26.0
